<a href="https://colab.research.google.com/github/dalenogare98/tcc/blob/master/codigo_metricas_salvandoarquivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Infos sistema

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [ ]:
!cat /proc/meminfo

MemTotal:       13333552 kB
MemFree:        10835156 kB
MemAvailable:   12539864 kB
Buffers:           73332 kB
Cached:          1779776 kB
SwapCached:            0 kB
Active:           640888 kB
Inactive:        1596464 kB
Active(anon):     352812 kB
Inactive(anon):      340 kB
Active(file):     288076 kB
Inactive(file):  1596124 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               136 kB
Writeback:             0 kB
AnonPages:        384388 kB
Mapped:           206152 kB
Shmem:               940 kB
Slab:             160296 kB
SReclaimable:     122676 kB
SUnreclaim:        37620 kB
KernelStack:        3780 kB
PageTables:         5320 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666776 kB
Committed_AS:    2641740 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:              928 kB
AnonHugePages:   

## Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import sys
import sklearn
import io
import random

## Carregando base de dados

Leitura dos csv da base NSL-KDD   s



In [ ]:
train_url = 'https://github.com/merteroglu/NSL-KDD-Network-Intrusion-Detection/raw/master/NSL_KDD_Train.csv'
#test_url = 'https://raw.githubusercontent.com/merteroglu/NSL-KDD-Network-Instrusion-Detection/master/NSL_KDD_Test.csv'

col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]


data_set = pd.read_csv(train_url,header=None, names = col_names)
#data_set = pd.read_csv('drive/My Drive/KDDTreino_projeto.csv',header=0, names = col_names)

#data_set_test = pd.read_csv(test_url, header=None, names = col_names)

print('Dimensions of the Training set:',data_set.shape)
#print('Dimensions of the Test set:',data_set_test.shape)

Dimensions of the Training set: (125973, 42)


Distribuição das classes nos conjuntos de dados

In [ ]:
print('Label distribution Training set:')
print(data_set['label'].value_counts())
print()
#print('Label distribution Test set:')
#print(data_set_test['label'].value_counts())

Label distribution Training set:
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: label, dtype: int64




# **Pré-processamento**

Visualizar dados antes do processamento


In [ ]:
data_set.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [ ]:
data_set.shape

(125973, 42)

## Agrupar ataques em (U2R, R2L, Probe, e DoS)

In [ ]:
condicao = [data_set['label'] == 'back',
            data_set['label'] == 'land',
            data_set['label'] == 'neptune',
            data_set['label'] == 'pod',
            data_set['label'] == 'smurf',
            data_set['label'] == 'teardrop',
            data_set['label'] == 'buffer_overflow',
            data_set['label'] == 'loadmodule',
            data_set['label'] == 'perl',
            data_set['label'] == 'rootkit',
            data_set['label'] == 'ftp_write',
            data_set['label'] == 'guess_passwd',
            data_set['label'] == 'imap',
            data_set['label'] == 'multihop',
            data_set['label'] == 'phf',
            data_set['label'] == 'spy',
            data_set['label'] == 'warezclient',
            data_set['label'] == 'warezmaster',
            data_set['label'] == 'ipsweep',
            data_set['label'] == 'nmap',
            data_set['label'] == 'portsweep',
            data_set['label'] == 'satan'
            ]

resultados = ['DoS', 'DoS', 'DoS', 'DoS', 'DoS', 'DoS', 
              'U2R', 'U2R', 'U2R', 'U2R', 
              'R2L', 'R2L', 'R2L', 'R2L', 'R2L', 'R2L', 'R2L', 'R2L',
              'probe', 'probe', 'probe', 'probe'
              ]

#utiliza bliblioteca numpy np
#seta na coluna label os resultados de acordo com as condições, os que nao tiver condiçção ficam como 'normal'
data_set['Attack'] = np.select(condicao, resultados, 'normal')
data_set.pop('label')
data_set.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,DoS
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


Distribuição das classes

In [ ]:
print('Label distribution Training set:')
print(data_set['Attack'].value_counts())
print()

Label distribution Training set:
normal    67343
DoS       45927
probe     11656
R2L         995
U2R          52
Name: Attack, dtype: int64



## Transformar dados categóricos

In [ ]:
from sklearn.preprocessing import LabelEncoder
import keras.preprocessing.text


columns_category = ['protocol_type','service','flag']

le = LabelEncoder()
for col in columns_category:
  le.fit(data_set[col])
  #self.classes[col]= le.classes_
  data_set[col] = le.fit_transform(data_set[col])
  #data_set_test[col] = le.fit_transform(df_test[col]) 

print("mostrar forma depois da categorização:")
data_set.head()

mostrar forma depois da categorização:


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack
0,0,1,20,9,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,2,44,9,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,DoS
3,0,1,24,9,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,1,24,9,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


## Tranformar classe em numerica

0 - normal, 1 - DoS, 2 - probe, 3 - R2L, 4 - U2R

In [ ]:
targets_data_set = data_set.pop('Attack')
#targets_data_set_test = data_set_test.pop('label')
print(targets_data_set)
targets_data_set_categorical = pd.Categorical(targets_data_set, ordered=True, categories=['normal', 'DoS', 'probe', 'R2L', 'U2R'])
#targets_data_set_test = pd.Categorical(targets_data_set_test)

#nome das classes na ordem 0 - normal, 1 - DoS, 2 - probe, 3 - R2L, 4 - U2R
nome_classes = ['normal', 'DoS', 'probe', 'R2L', 'U2R']

targets_data_set_categorical.categories
#targets_data_set_test.categories
print(targets_data_set_categorical.codes)

data_set['Attack'] = targets_data_set_categorical.codes
#data_set_test['label'] = targets_data_set_test.codes

0         normal
1         normal
2            DoS
3         normal
4         normal
           ...  
125968       DoS
125969    normal
125970    normal
125971       DoS
125972    normal
Name: Attack, Length: 125973, dtype: object
[0 0 1 ... 0 1 0]


Distribuição das classes

In [ ]:
print('Label distribution Training set:')
print(data_set['Attack'].value_counts())
print()

Label distribution Training set:
0    67343
1    45927
2    11656
3      995
4       52
Name: Attack, dtype: int64



Mostrar dados com a classe numérica

In [ ]:
data_set.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack
0,0,1,20,9,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,0
1,0,2,44,9,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0
2,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1
3,0,1,24,9,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0
4,0,1,24,9,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0


## Separar dados das classes

In [ ]:

back_up = data_set[:]
train_targ = back_up.pop('Attack')
train_feat = back_up

back_up.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,1,20,9,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00
1,0,2,44,9,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00
2,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00
3,0,1,24,9,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01
4,0,1,24,9,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
train_targ.head()

0    0
1    0
2    1
3    0
4    0
Name: Attack, dtype: int8

# DNN

Bibliotecas

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.callbacks import EarlyStopping
from keras.callbacks import CSVLogger
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from sklearn import preprocessing
import time
from sklearn.metrics import multilabel_confusion_matrix


Definindo modelo do classificador

In [ ]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(82, activation='relu', input_shape=[41]),
    keras.layers.Dense(82, activation='relu'),
    keras.layers.Dense(82, activation='relu'),
    keras.layers.Dense(82, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
  ])

  return model

**Cross** **validation**

In [ ]:
nome_classes = ['normal', 'DoS', 'probe', 'R2L', 'U2R']
num_class = 5
num_folds = 10

conjunto_dados = train_feat.values
conjunto_dados_classes = train_targ


model = build_model()

test_time_per_fold = []
matriz_per_fold = []
acc_geral_per_fold = []
precision_geral_per_fold = []

kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no = 1
for train, test in kfold.split(conjunto_dados, conjunto_dados_classes):
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  model = build_model()
  print(model.summary())
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  csv_logger = CSVLogger('training.log')
  #funcao para interromper treinamento quando o erro for suficientemente pequeno
  early_stopping = EarlyStopping(monitor='loss', patience=20)

  fit = model.fit(conjunto_dados[train], conjunto_dados_classes[train], epochs=100, verbose=2, callbacks=[early_stopping])

  test_time_start = time.time()

  predictions_dnn = model.predict_classes(conjunto_dados[test])

  test_time = time.time() - test_time_start

  print('tempo teste: ')
  print(test_time)

  acc = sklearn.metrics.accuracy_score(conjunto_dados_classes[test], predictions_dnn)
  acc_geral_per_fold.append(acc)
  print(acc)

  precision = sklearn.metrics.precision_score(conjunto_dados_classes[test], predictions_dnn, average='weighted')
  precision_geral_per_fold.append(precision)
  print(precision)


  
  print(conjunto_dados_classes[test])
  print(predictions_dnn)
  m = multilabel_confusion_matrix(conjunto_dados_classes[test], predictions_dnn)
  matriz_per_fold.append(m)
  
  print(f'Score for fold {fold_no}: ')
  print('Accuracy: {}'.format(acc))

  test_time_per_fold.append(test_time)
  
  # Increase fold number
  fold_no = fold_no + 1

Salvar resultados em arquivo

In [ ]:
acc_per_fold = []
precision_per_fold = []
recall_per_fold = []
TNR_per_fold = []
f1_score_per_fold = []

print('Salvando resultados em arquivo...')

#caminho e nome do arquivo, definir conforme seu computador
arquivo = open('./drive/My Drive/Colab Notebooks/resultado100epochs/4C82NREFEITO', 'w') 
texto = ' ----------------- RESULTADOS DO EXPERIMENTO ----------------- \n'
#tempo médio para execução do teste
texto += f'TEMPO MÉDIO: {np.mean(test_time_per_fold)}\n'
#Media acuracia independente de classe 
texto += f'ACURACIA GERAL MEDIA (independente de classe): {np.mean(acc_geral_per_fold)}\n'
texto += f'PRECISION GERAL MEDIA (independente de classe): {np.mean(precision_geral_per_fold)}\n'
texto += '\n\n\n\n\n'

texto += 'RESULTADOS POR FOLD DO CROSS VALIDATION E POR CLASSE\n\n'
for i in range(0, num_folds):
  acc_classes = []
  precision_classes = []
  recall_classes = []
  TNR_classes = []
  f1_score_classes = []
  texto += f'\n\nFold {i+1}:\n'
  texto += '=====================================================================\n\n'
  #texto += f'Acurácia média do fold: {np.mean(acc_per_fold)}\n'  
  texto += f'Tempo do fold: {test_time_per_fold[i]}\n'
  texto += f'Acurácia do fold: {acc_geral_per_fold[i]}'
  texto += f'Precision do fold: {precision_geral_per_fold[i]}'
  for j in range(0, num_class):
    texto += f'Classe : {j}\n'

    #acessa a matriz do fold i
    matriz = matriz_per_fold[i]

    #separa a matriz de cada classe j em tn, fp, fn, tp  
    print(matriz[j])
    tn = matriz[j][0][0]
    fp = matriz[j][0][1]
    fn = matriz[j][1][0]
    tp = matriz[j][1][1]

    #imprime matriz no arquivo
    texto += '\n\n-- N --|-- P --\n'
    texto += f'N| {tn} | {fp} |\n'
    texto += '-----------------------\n'
    texto += f'P| {fn} | {tp} |\n'
    texto += '\n\n'

    #calcula as métricas com base nos dados tp, tn, fn, fp
    acc_classes.append(((tn+tp)/(tn+tp+fn+fp)))
    precision = (tp/(tp+fp))
    precision_classes.append(precision)
    recall = (tp/(tp+fn))
    recall_classes.append(recall)
    TNR_classes.append((tn/(tn+fp)))
    f1_score_classes.append((2 * ((precision * recall)/(precision + recall))))

    #imprimir linha no arquivo com as métricas calculadas para a classe j no fold i
    texto += '             acc,                     loss,                   precision,            recall,                TNR,              f1-score\n'
    texto += f'Classe {j}:  {acc_classes[j]},   {1-acc_classes[j]},   {precision_classes[j]},   {recall_classes[j]},     {TNR_classes[j]},  {f1_score_classes[j]}\n'

    acc_per_fold.append(acc_classes)
    precision_per_fold.append(precision_classes)
    recall_per_fold.append(recall_classes)
    TNR_per_fold.append(TNR_classes)
    f1_score_per_fold.append(f1_score_classes)



#imprimir no arquivo os resultados referentes a todas as execuções do cross_validation, é feita a média de cada métrica para cada classe
texto += '\n\n\n\n\n============================================================================\n'
texto += 'RESULTADOS FINAIS POR CLASSE, MÉDIA DAS EXECUÇÕES DO CROSS: \n\n\n'

texto += ' Classe :       acc,                  loss,             precision,           recall,              TNR,              f1-score\n'


#for para percorrer os vetores e calcular a média das métricas dos 10 folds
for i in range(0,num_class):
  soma_acc = 0
  soma_precision = 0
  soma_recall = 0
  soma_TNR = 0
  soma_f1_score = 0

  for j in range(0, num_folds):
      soma_acc +=  acc_per_fold[j][i]
      soma_precision +=  precision_per_fold[j][i]
      soma_recall += recall_per_fold[j][i]
      soma_TNR += TNR_per_fold[j][i]
      soma_f1_score += f1_score_per_fold[j][i]

  texto += f' {nome_classes[i]} : {soma_acc/num_folds}, {1-(soma_acc/num_folds)}, {soma_precision/num_folds}, {soma_recall/num_folds}, {soma_TNR/num_folds}, {soma_f1_score/num_folds}\n'

texto += '\n\n\n\n\n'
arquivo.write(texto) 
arquivo.close()
print('------------------------------------------------------------------------')

Salvando resultados em arquivo...
[[5664  315]
 [  10 6609]]
[[7884    0]
 [ 137 4577]]
[[11446     4]
 [  123  1025]]
[[12486     7]
 [   44    61]]
[[12586     0]
 [   12     0]]
[[5826   49]
 [1081 5642]]
[[7388  598]
 [ 195 4417]]
[[10675   744]
 [   49  1130]]
[[12502    18]
 [   78     0]]
[[12592     0]
 [    6     0]]
[[5774  106]
 [ 762 5956]]
[[6991  971]
 [  38 4598]]
[[11405    47]
 [  304   842]]
[[12437    69]
 [   84     8]]
[[12591     1]
 [    6     0]]
[[5521  313]
 [ 254 6509]]
[[7385  622]
 [ 117 4473]]
[[11254   204]
 [  663   476]]
[[12498     0]
 [   99     0]]
[[12591     0]
 [    6     0]]
[[5702  192]
 [1280 5423]]
[[7889  147]
 [ 301 4260]]
[[10872   494]
 [  128  1103]]
[[11520   976]
 [   99     2]]
[[12596     0]
 [    1     0]]
[[5661  239]
 [ 378 6319]]
[[7722  263]
 [ 119 4493]]
[[11245   172]
 [   74  1106]]
[[12490     5]
 [  102     0]]
[[12591     0]
 [    6     0]]
[[5623  171]
 [ 173 6630]]
[[7946  119]
 [  34 4498]]
[[11357    79]
 [   70  1091]]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars
